# Using DFTpy mixer

In [1]:
import _qepy
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
        'input_dft' : "'PBE'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

Iter:  0  - Conv:  0.07423450252710213
Iter:  1  - Conv:  0.0014686764583955348
Iter:  2  - Conv:  3.4654773793272354e-05
Iter:  3  - Conv:  6.680306265302417e-08
Iter:  4  - Conv:  2.5412863642447476e-09


-137.91449178319718

In [7]:
from dftpy.functional import Hartree

In [8]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.2.out')

In [9]:
rho = driver.get_density().copy()
coef = 0.7
for i in range(10):
    driver.diagonalize()
    #
    rho_new = driver.get_density().copy()
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    print ('Iter: ',i,' - Conv: ', error)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = (1-coef)*rho + coef * rho_new
    driver.set_density(rho)
driver.calc_energy()

Iter:  0  - Conv:  0.03712146707277951
Iter:  1  - Conv:  0.0007346997237828299
Iter:  2  - Conv:  3.914394446485253e-05
Iter:  3  - Conv:  3.3366355611395276e-06
Iter:  4  - Conv:  3.240568487976684e-07
Iter:  5  - Conv:  3.072301214068241e-08
Iter:  6  - Conv:  2.813491299355543e-09


-137.914491783489

In [10]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.3.out')

In [11]:
from dftpy.mixer import Mixer
driver.mixer = Mixer(scheme='pulay')

In [12]:
rho = driver.data2field(driver.get_density().copy())
for i in range(10):
    driver.diagonalize()
    #
    rho_new = driver.data2field(driver.get_density().copy())
    error = Hartree.compute(rho_new-rho).energy
    print ('Iter: ',i,' - Conv: ', error)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = driver.mixer(rho, rho_new, coef=0.7)
    density = driver.field2data(rho)
    driver.set_density(density)
driver.calc_energy()

Iter:  0  - Conv:  0.037130390200344775
Iter:  1  - Conv:  0.0007344758020421512
Iter:  2  - Conv:  1.3112210595090678e-05
Iter:  3  - Conv:  6.112684999094081e-08
Iter:  4  - Conv:  3.238350933236666e-10


-137.91449178172286

In [13]:
ions = driver.get_dftpy_ions()

In [14]:
rho.write('tmp.xsf', ions=ions)